## Load data and functions.

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from implementations import *
from proj1_helpers import *
from myhelpers import *

In [2]:
data_path = "datasets/train.csv"
yb, input_data, ids = load_csv_data(data_path, sub_sample=False)

## Split according to jet_number.

In [97]:
# getting the number of data of PRI_jet_num = 0, 1, 2, 3
def class_allocation(input_data, yb, ids):
    num_0 = 0
    num_1 = 0
    num_2 = 0
    num_3 = 0 
    for j in range(input_data.shape[0]):
        if input_data[j,22] == 0:
            num_0 = num_0 + 1
        elif input_data[j,22] == 1:
            num_1 = num_1 + 1
        elif input_data[j,22] == 2:
            num_2 = num_2 + 1
        elif input_data[j,22] == 3:
            num_3 = num_3 + 1

    # initializing arrays for different PRI_jet_num

    size = (num_0, input_data.shape[1])
    jet_num_0 = np.zeros(size)
    ids_0 = np.zeros(num_0)
    y0 = np.zeros(num_0)

    size = (num_1, input_data.shape[1])
    jet_num_1 = np.zeros(size)
    ids_1 = np.zeros(num_1)
    y1 = np.zeros(num_1)
    
    size = (num_2, input_data.shape[1])
    jet_num_2 = np.zeros(size)
    ids_2 = np.zeros(num_2)
    y2 = np.zeros(num_2)
    
    size = (num_3, input_data.shape[1])
    jet_num_3 = np.zeros(size)
    ids_3 = np.zeros(num_3)
    y3 = np.zeros(num_3)
    
    # allocating input_data to different classes due to their PRI_jet_num value
    i_0, i_1, i_2, i_3 = 0, 0, 0, 0


    for i in range(input_data.shape[0]):
        if input_data[i,22] == 0:
            jet_num_0[i_0,:] = input_data[i,:]
            ids_0[i_0] = i
            y0[i_0] = yb[i]
            i_0 = i_0 + 1
            
        elif input_data[i,22] == 1:
            jet_num_1[i_1,:] = input_data[i,:]
            ids_1[i_1] = i
            y1[i_1] = yb[i]
            i_1 = i_1 + 1
            
        elif input_data[i,22] == 2:
            jet_num_2[i_2,:] = input_data[i,:]
            ids_2[i_2] = int(i)
            y2[i_2] = yb[i]
            i_2 = i_2 + 1
            
        elif input_data[i,22] == 3:
            jet_num_3[i_3,:] = input_data[i,:]
            ids_3[i_3] = int(i)
            y3[i_3] = yb[i]
            i_3 = i_3 + 1
            
    ids_0 = ids_0.astype(int)
    ids_1 = ids_1.astype(int)
    ids_2 = ids_2.astype(int)
    ids_3 = ids_3.astype(int)
    
    return jet_num_0, y0, ids_0, jet_num_1, y1, ids_1, jet_num_2, y2, ids_2, jet_num_3, y3, ids_3

In [15]:
jet0_in, yb0, ids0, jet1_in, yb1, ids1, jet2_in, yb2, ids2, jet3_in, yb3, ids3 = class_allocation(input_data, yb, ids)

In [16]:
print(jet0_in.shape, jet1_in.shape, jet2_in.shape, jet3_in.shape)

(99913, 30) (77544, 30) (50379, 30) (22164, 30)


Remove the feature "pri_jet_num" since we've already taken into account its contribute

In [17]:
jet0_in = np.delete(jet0_in,22,1)
jet1_in = np.delete(jet1_in,22,1)
jet2_in = np.delete(jet2_in,22,1)
jet3_in = np.delete(jet3_in,22,1)

In [18]:
print(jet0_in.shape, jet1_in.shape, jet2_in.shape, jet3_in.shape)

(99913, 29) (77544, 29) (50379, 29) (22164, 29)


## Cleaning and standardizing data.

In [19]:
# We clean the four matrices from features with a standard deviation of 0. These features are considered non-measured.

def remove_useless_cols(matrix):
    std_cols = np.std(matrix, axis=0)
    num_rem = 0
    for col in reversed(range(matrix.shape[1])):
        if std_cols[col] == 0:
            num_rem = num_rem + 1
    #removed_cols = np.zeros(num_rem)
    #k = 0        
    for col in reversed(range(matrix.shape[1])):
        if std_cols[col] == 0:
            matrix = np.delete(matrix,col,1)
            #removed_cols[k] = col
            #k = k + 1
    return matrix

In [20]:
jet0_in = remove_useless_cols(jet0_in)
jet1_in = remove_useless_cols(jet1_in)
jet2_in = remove_useless_cols(jet2_in)
jet3_in = remove_useless_cols(jet3_in)

In [21]:
print(jet0_in.shape, jet1_in.shape, jet2_in.shape, jet3_in.shape)

(99913, 18) (77544, 22) (50379, 29) (22164, 29)


In [22]:
# Remove the invalid values -999 such that they are 0 after the standardization

def clean_data(matrix):
    cleansubx = matrix
    cleansubx[np.where(cleansubx == -999)] = 0
    means_by_columns = np.mean(cleansubx, axis=0)
    for i in range(matrix.shape[1]):
        matrix[:,np.where(matrix[:,i]==-999)] = means_by_columns[i]
    return matrix

def clean_and_standardize(matrix):
    matrix = clean_data(matrix)
    matrix = standardize(matrix)
    return matrix

In [23]:
jet0_clean = clean_and_standardize(jet0_in)
jet1_clean = clean_and_standardize(jet1_in)
jet2_clean = clean_and_standardize(jet2_in)
jet3_clean = clean_and_standardize(jet3_in)

In [27]:
print(jet0_clean.shape, jet1_clean.shape, jet2_clean.shape, jet3_clean.shape)

(99913, 18) (77544, 22) (50379, 29) (22164, 29)


## Eliminate correlations in each group.

In [28]:
# Plot the correlation matrix.

def plot_corr(matrix):
    cols = matrix.shape[1]
    cor_matrix = np.corrcoef(matrix.T)
    plt.figure(1, figsize=(12,12))
    plt.matshow(np.abs(cor_matrix),1)
    plt.xticks(range(cols), range(cols))
    plt.yticks(range(cols), range(cols))
    plt.colorbar()
    #plt.savefig('correlation_matrix'+'.png')

In [29]:
# Select the correlated columns 

def get_correlations(matrix, bound):
    high_corr = []
    cor_matrix = np.corrcoef(matrix.T)
    for i in range(cor_matrix.shape[0]):
        for j in range(i+1,cor_matrix.shape[1]):
            if cor_matrix[i,j] > bound:
                high_corr.append((i,j))
    return high_corr

In [30]:
bound = 0.8
print(get_correlations(jet0_clean, bound))
print(get_correlations(jet1_clean, bound))
print(get_correlations(jet2_clean, bound))
print(get_correlations(jet3_clean, bound))

[(3, 5), (6, 9)]
[(3, 6), (3, 18), (3, 21), (6, 18), (6, 21), (18, 21)]
[(3, 9), (4, 5), (9, 21), (9, 22), (9, 28), (21, 28), (22, 28)]
[(9, 21), (9, 22), (9, 28), (21, 22), (21, 28), (22, 28), (25, 28)]


In [31]:
# Remove the selected columns

jet0 = np.delete(jet0_clean,[5,9],1)
jet1 = np.delete(jet1_clean,[6, 18, 21],1)
jet2 = np.delete(jet2_clean,[9, 5, 21, 22, 28],1)
jet3 = np.delete(jet3_clean,[21, 22, 28],1)

print('Size jet0 : {s}'.format(s=jet0.shape))
print('Size jet1 : {s}'.format(s=jet1.shape))
print('Size jet2 : {s}'.format(s=jet2.shape))
print('Size jet3 : {s}'.format(s=jet3.shape))

Size jet0 : (99913, 16)
Size jet1 : (77544, 19)
Size jet2 : (50379, 24)
Size jet3 : (22164, 26)


## Defining helper functions.

In [32]:
# BUILDING POLYNOMIAL BASIS

# Function that builds a polynomial basis of the chosen degree for a column 
def build_poly_col(x, degree):
    """polynomial basis functions for input column x, for j=1 up to j=degree."""
    
    y = x        
    for n in range(2,degree+1):
        x = np.c_[x, np.power(y,n)]
            
    return x

# Function that builds a polynomial basis for the data (each column has the same degree)
def build_poly(data, degree):
    """polynomial basis for input data up to the chosen degree in each column"""
    
    X = np.c_[np.ones(data.shape[0])]
    for j in range(data.shape[1]):
        x_col = build_poly_col(data[:,j], degree)
        X = np.c_[X, x_col]
        
    return X


In [33]:
# SPLITTING THE TRAINING SET FOR CROSS VALIDAITON

# Functions to build the indices and split the data to perform cross validation

def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)


def split_dataset(y, x, k, k_indices):
    """Returns the matrices and vectors Test and Train used in a k-fold cross validation"""
    
    # get k'th subgroup in test, others in train.
    X_test = x[k_indices[k]]
    y_test = y[k_indices[k]]
    X_train = np.delete(x,k_indices[k],0)
    y_train = np.delete(y,k_indices[k])
    
    return X_test, y_test, X_train, y_train


## Grid search Ridge regression.
We look for the best lambda and polynomial degree for each group trough cross validation, saving the weights too. The tool to evaluate the model is the accuracy but we compute the rmse too. 

In [34]:
def ridge_deg_lam(yb, data, kfolds, degree, lambda_):
    """Builds a polynomial basis according to the chosen degree and solves the ridge regression
        with the normal equations for lambda_, performing a k-fold cross validation 
        to compute the rmse and the weights"""
    
    rmse_train=[]
    rmse_test=[]
    seed = 3

    # Build indices to split data
    k_indices = build_k_indices(yb, kfolds, seed)

    rmse_tr_k = []
    rmse_te_k = []
    w_ridge_k = []
    accuracy_k = []
        
    # Build X matrix
    data1 = build_poly(data, degree)
    
    # Loop over the folds
    for k in range(kfolds):
        
        data_test, y_test, data_train, y_train = split_dataset(yb, data1, k, k_indices)   
        
        # Train the model
        w_k, mse_k = ridge_regression(y_train, data_train, lambda_)
        rmse_test = np.sqrt(2*compute_mse(y_test, data_test, w_k))
            
        # Do the prediction on the current test set and compute accuracy
        ypred = predict_labels(w_k, data_test)
        acc_k = compute_accuracy(ypred, y_test)
            
        # Store rmse, weights and accuracy
        rmse_tr_k.append(np.sqrt(2*mse_k))
        rmse_te_k.append(rmse_test)
        w_ridge_k.append(w_k)
        accuracy_k.append(acc_k)
        
    # Compute the means over the folds
    rmse_tr = np.mean(rmse_tr_k)
    rmse_te = np.mean(rmse_te_k)
    w_ridge = np.mean(w_ridge_k, axis=0)
    accuracy = np.mean(accuracy_k)
        

    return rmse_tr, rmse_te, w_ridge, accuracy

In [35]:
# Visualize the error and accuracy plots

def ridge_lambda_vis(lambdas, rmse_tr, rmse_te, std_train, std_test, acc, deg):
    """visualization of the curves of rmse_tr and rmse_te, and of the accuracy"""
    
    plt.figure(4, figsize=(16,5))
    plt.subplot(1,2,1)
    plt.semilogx(lambdas, rmse_tr, marker=".", color='b', label='Train error')
    plt.semilogx(lambdas, rmse_te, marker=".", color='r', label='Test error')
    #plt.errorbar(np.log10(lambdas), rmse_tr, std_train, label='Train error')
    #plt.errorbar(np.log10(lambdas), rmse_te, std_test, label='Test error')
    #plt.plot(lambdas, rmse_tr, marker=".", color='b', label='Train error')
    #plt.plot(lambdas, rmse_te, marker=".", color='r', label='Test error')
    plt.xlabel("Lambda")
    plt.ylabel("RMSE")
    plt.title("RMSE ridge regression with degree {ddd}".format(ddd=deg))
    plt.legend(loc=2)
    plt.grid(True)
    
    plt.subplot(1,2,2)
    plt.semilogx(lambdas, acc,  marker=".", color='b', label='Accuracy')
    #plt.plot(lambdas, acc,  marker=".", color='b', label='Accuracy')
    plt.xlabel("Lambda")
    plt.ylabel("Accuracy")
    plt.title("ACCURACY ridge regression with degree {ddd}".format(ddd=deg))
    plt.legend(loc=1)
    plt.grid(True)

In [ ]:
# Trying with a fixed degree 
lambdas = 0.05
deg = 6
rmse_tr0, rmse_te0, weight0, acc0 = ridge_deg_lam(yb0, jet0, 4, deg, lambdas)
#ridge_lambda_vis(lambdas, rmse_tr0, rmse_te0, _, _, acc0, deg)

In [ ]:
print(rmse_te0)

### Doing grid search for each class.

In [43]:
# Grid search
def grid_search_ridge(y, tx, kfolds, deg_grid, lam_grid):
    """Performs a grid search over degrees and lambdas to find the best parameters for the model trained over tx
       Returns the optimal lambda, the optimal degree and the optimal weights obtained with a kfolds cross validation
       (The tool to evaluate and compare models is the accuracy)"""
    
    losses = np.zeros((len(deg_grid),len(lam_grid)))
    accuracies = np.zeros((len(deg_grid),len(lam_grid)))
    acc_old = -1
    
    # Perform a grid search
    for i, deg in enumerate(deg_grid):
        for j, lam in enumerate(lam_grid):  
            
            rmse_train, rmse_test, w_ridge, acc = ridge_deg_lam(y, tx, kfolds, deg, lam)
            
            losses[i,j] = rmse_test
            accuracies[i,j] = acc
            
            if acc > acc_old:
                weight_opt = w_ridge
                deg_opt_acc = deg     # it' better to select the optimal parameters in this way (not with get_best()) because if
                lam_opt_acc = lam     # the accuracy is the same with different parameters, we may get different sizes of the 
                max_accuracy = acc    # weights vector and the matrix with the polynomial basis
            
            acc_old = acc            
        
    # Get best parameters
    min_rmse_test, deg_opt_err, lam_opt_err = get_best_rmse(lambdas_grid, degree_grid, losses)  
    #max_accuracy, deg_opt_acc, lam_opt_acc = get_best_accuracy(lam_grid, deg_grid, accuracies)
    
    return min_rmse_test, deg_opt_err, lam_opt_err, max_accuracy, deg_opt_acc, lam_opt_acc, weight_opt

In [47]:
# Define the range for the parameters 
degree_grid = range(1,12)
lambdas_grid = np.logspace(-7,2,50)
kfolds = 4

# n_columns jet0 = 16
rmse0, degerr0, lamerr0, acc0, degacc0, lamacc0, weights0 = grid_search_ridge(yb0, jet0, kfolds, degree_grid, lambdas_grid)

print('Minimum Test RMSE: {rmse}'.format(rmse=rmse0))
print('Optimal degree: {d}'.format(d=degerr0))
print('Optimal lambda: {l}'.format(l=lamerr0))

print('Maximum accuracy: {acc}'.format(acc=acc0))
print('Optimal degree: {d}'.format(d=degacc0))
print('Optimal lambda: {l}'.format(l=lamacc0))
print('Weights shape: {w}'.format(w=weights0.shape))

Minimum Test RMSE: 0.737804793602948
Optimal degree: 1
Optimal lambda: 0.0007196856730011522
Maximum accuracy: 0.7950996877251981
Optimal degree: 11
Optimal lambda: 7.9060432109077015
Weights shape: (177,)


In [44]:
# n_columns jet1 = 19
rmse1, degerr1, lamerr1, acc1, degacc1, lamacc1, weights1 = grid_search_ridge(yb1, jet1, kfolds, degree_grid, lambdas_grid)

print('Minimum Test RMSE: {rmse}'.format(rmse=rmse1))
print('Optimal degree: {d}'.format(d=degerr1))
print('Optimal lambda: {l}'.format(l=lamerr1))

print('Maximum accuracy: {acc}'.format(acc=acc1))
print('Optimal degree: {d}'.format(d=degacc1))
print('Optimal lambda: {l}'.format(l=lamacc1))
print('Weights shape: {w}'.format(w=weights1.shape))

Minimum Test RMSE: 0.8080010143389709
Optimal degree: 5
Optimal lambda: 0.03237457542817647
Maximum accuracy: 0.8033761477354793
Optimal degree: 11
Optimal lambda: 0.00047148663634573947
Weights shape: (210,)


In [45]:
# n_columns jet2 = 24
rmse2, degerr2, lamerr2, acc2, degacc2, lamacc2, weights2 = grid_search_ridge(yb2, jet2, kfolds, degree_grid, lambdas_grid)

print('Minimum Test RMSE: {rmse}'.format(rmse=rmse2))
print('Optimal degree: {d}'.format(d=degerr2))
print('Optimal lambda: {l}'.format(l=lamerr2))

print('Maximum accuracy: {acc}'.format(acc=acc2))
print('Optimal degree: {d}'.format(d=degacc2))
print('Optimal lambda: {l}'.format(l=lamacc2))
print('Weights shape: {w}'.format(w=weights2.shape))

Minimum Test RMSE: 0.7959194689173034
Optimal degree: 4
Optimal lambda: 0.021209508879201925
Maximum accuracy: 0.829958710497062
Optimal degree: 11
Optimal lambda: 0.0025595479226995384
Weights shape: (265,)


In [46]:
# n_columns jet3 = 26
rmse3, degerr3, lamerr3, acc3, degacc3, lamacc3, weights3 = grid_search_ridge(yb3, jet3, kfolds, degree_grid, lambdas_grid)

print('Minimum Test RMSE: {rmse}'.format(rmse=rmse3))
print('Optimal degree: {d}'.format(d=degerr3))
print('Optimal lambda: {l}'.format(l=lamerr3))

print('Maximum accuracy: {acc}'.format(acc=acc3))
print('Optimal degree: {d}'.format(d=degacc3))
print('Optimal lambda: {l}'.format(l=lamacc3))
print('Weights shape: {w}'.format(w=weights3.shape))

Minimum Test RMSE: 0.7987655327075097
Optimal degree: 4
Optimal lambda: 0.0025595479226995384
Maximum accuracy: 0.8290019852012271
Optimal degree: 11
Optimal lambda: 0.00047148663634573947
Weights shape: (287,)


## Load, split, clean, build test data.

In [98]:
data_path = "datasets/test.csv"
yb_test, input_data_test, ids_test = load_csv_data(data_path, sub_sample=False)

In [99]:
# Split

test0_in, test_yb0, test_ids0, test1_in, test_yb1, test_ids1, test2_in, test_yb2, test_ids2, test3_in, test_yb3, test_ids3 = class_allocation(input_data_test, yb_test, ids_test)

print(test0_in.shape, test1_in.shape, test2_in.shape, test3_in.shape)

(227458, 30) (175338, 30) (114648, 30) (50794, 30)


In [100]:
print(ids_test[1:20])
print(test_ids0[:10])
print(test_ids1[:10])
print(test_ids2[:10])
print(test_ids3[:10])
print( test_ids0[:10].astype(int))

[350001 350002 350003 350004 350005 350006 350007 350008 350009 350010
 350011 350012 350013 350014 350015 350016 350017 350018 350019]
[ 0  2  3  5  8 11 12 14 15 16]
[ 1  6 10 18 20 23 26 31 33 34]
[ 7  9 13 21 24 43 47 53 62 64]
[  4  22  25  28  35  71  85  99 118 131]
[ 0  2  3  5  8 11 12 14 15 16]


In [101]:
# We clean the four matrices from features with a standard deviation of 0. These features are considered non-measured.

test0_in = remove_useless_cols(test0_in)
test1_in = remove_useless_cols(test1_in)
test2_in = remove_useless_cols(test2_in)
test3_in = remove_useless_cols(test3_in)

print(test0_in.shape, test1_in.shape, test2_in.shape, test3_in.shape)

(227458, 18) (175338, 22) (114648, 29) (50794, 29)


In [102]:
# Clean the invalid values -999 and standardize
test0_clean = clean_and_standardize(test0_in)
test1_clean = clean_and_standardize(test1_in)
test2_clean = clean_and_standardize(test2_in)
test3_clean = clean_and_standardize(test3_in)

In [103]:
# Eliminate correlated columns

test0 = np.delete(test0_clean,[5,9],1)
test1 = np.delete(test1_clean,[6, 18, 21],1)
test2 = np.delete(test2_clean,[9, 5, 21, 22, 28],1)
test3 = np.delete(test3_clean,[21, 22, 28],1)


## Apply the model and make predictions.

In [104]:
# Build the polynomial basis according to the groups
test0fin = build_poly(test0, degacc0)
test1fin = build_poly(test1, degacc1)
test2fin = build_poly(test2, degacc2)
test3fin = build_poly(test3, degacc3)

print('Size jet0 : {s}, size weights0 : {w}'.format(s=test0fin.shape, w=weights0.shape))
print('Size jet1 : {s}, size weights1 : {w}'.format(s=test1fin.shape, w=weights1.shape))
print('Size jet2 : {s}, size weights2 : {w}'.format(s=test2fin.shape, w=weights2.shape))
print('Size jet3 : {s}, size weights3 : {w}'.format(s=test3fin.shape, w=weights3.shape))

Size jet0 : (227458, 177), size weights0 : (177,)
Size jet1 : (175338, 210), size weights1 : (210,)
Size jet2 : (114648, 265), size weights2 : (265,)
Size jet3 : (50794, 287), size weights3 : (287,)


In [105]:
ypred0 = predict_labels(weights0, test0fin)
ypred1 = predict_labels(weights1, test1fin)
ypred2 = predict_labels(weights2, test2fin)
ypred3 = predict_labels(weights3, test3fin)

create_submission(ypred0, test_ids0, ypred1, test_ids1, ypred2, test_ids2, ypred3, test_ids3, ids_test, 'ridge_splitted.csv')

[ 0  2  3  5  8 11]
